<div class="alert alert-info" style="font-family:'arial';font-size:30px"> Genomic Data Fundamentals </div>

02_Hail_part2_GWAS

**Authors:** Jennifer Zhang, Francis Ratsimbazafy, Jun Qian

**Contributors:** Christopher Lord, Nicole Deflaux, Kelsey Mayo, Lee Lichtenstein, *All of Us* Genomic Users

*With minor modifications from Stephan Cordogan

<div class="alert alert-block alert-danger" style="font-size:16px">
Before you start running this notebook, make sure you are using a Dataproc runtime. To do so,<br>
   <ul>
       <li> Click on the <b>Jupyter</b> icon on the right hand side of the screen </li>
       <li> Inside <b>Recommended environments</b>, select <b>Hail Genomics Analysis</b> which creates the computer type <b>Dataproc Cluster</b> </li>
       <li> Select reasonable defaults for CPU, RAM, disk and numbers of workers </li>
       <li> Click on <b>Next</b> </li>
    </ul>
</div>

- The cell below provides the environment information, time, and cost to run the notebook. 


- You can increase the number of workers to make this job complete faster.


- You can also run this notebook in the background. Please refer to the notebook [Run Notebooks in the Background](https://workbench.researchallofus.org/workspaces/aou-rw-87f97daa/howtoworkwithallofusgenomicdatahailplink/notebooks/preview/05_How%20to%20Run%20Notebooks%20in%20the%20Background.ipynb) for more details.

<div class="alert alert-block alert-info">
    
<li> Main node: 8CPUs, 30GB RAM, 100 GB Disk</li>

<li> Workers (50/50): 4CPUs, 15GB RAM, 150GB Disk</li>

<li> Time and Cost: \$16.86/h, ~8min, \$2.24</li>

</div>

# Objectives

We recommend that all users new to genomic data manipulation / analysis read this notebook to learn the basics of Hail.

**The 1 min summary?** This is the second and final part of the tutorial series on Hail. 

In this notebook, you will learn 
- how to load the previously saved phenotype
- how these two data types are merged into one matrix table for the GWAS modeling
- how to use Hail to run your GWAS model 
- how to create Manhattan and QQ-plot to check your results.

Before starting our analysis, we first need to import the following packages and environment variables:

In [ ]:
from datetime import datetime
import os
import pandas as pd

In [ ]:
start = datetime.now()

We recommend saving the workspace bucket path as a variable to read and save files.

In [ ]:
bucket = os.getenv('WORKSPACE_BUCKET')
bucket

# Working with genotype data with Hail

## Initialize Hail

We first need to import Hail. We then need to import Bokeh, which Hail uses for visualization.

In [ ]:
import hail as hl
from hail.plot import show
from bokeh.plotting import output_file, save
import bokeh.io
from bokeh.io import *
from bokeh.resources import INLINE
#bokeh.io.output_notebook(INLINE) 
%matplotlib inline

The *All of U*s genomic data use "hg38" as reference, so we set the default reference "GRCh38" while initializing Spark.

In [ ]:
hl.init(default_reference = "GRCh38")

## Loading the Hail MatrixTable containing the variants

The *All of Us* Data and Research Center provides eight Hail MatrixTables with variants from different genome regions. In this tutorial, we use the Hail MatrixTable ClinVar variants. 
There are two versions of the Hail MatrixTable for each region: a MatrixTable with multi-allelic variants and a MatrixTable with multi-allelic variants split. We will use the latter version in this tutorial as Hail will only consider the first alternative allele in the regression analysis.

Options are also given for the ACAF version

In [ ]:
mt_path = os.getenv("WGS_CLINVAR_SPLIT_HAIL_PATH")
# mt_path = os.getenv("WGS_ACAF_THRESHOLD_SPLIT_HAIL_PATH")
# mt_path = os.getenv("WGS_ACAF_THRESHOLD_MULTI_HAIL_PATH")

mt_path

In the following cell, we will import the Hail MatrixTable.

In [ ]:
mt = hl.read_matrix_table(mt_path)

In [ ]:
mt.count()

<div class="alert alert-block alert-danger">
To speed up execution time and reduce costs, we will only work on a small interval of the genomic data. 
</div>



You can also specify a small region within a single chromosome. See examples of how to specify genomic region intervals in the [Hail documentation](https://hail.is/docs/0.1/representation/hail.representation.Interval.html). **This part is not necessary in real genome-wide association studies.**

In [ ]:
test_intervals = ['chr21']

In [ ]:
# mt = hl.filter_intervals(
#     mt,
#     [hl.parse_locus_interval(x,)
#      for x in test_intervals])

In [ ]:
# mt.count()

## Load phenotypic data

Let’s read the pre-generated phenotypic data into a Hail table for later use. We will use the the function `import_table` in this step.

In the demographics dataframe that we created and saved to the bucket earlier, each row represents data for one person ID. The same person IDs also represent the columns in the matrix table.

- Read the phenotype file from your workspace bucket, if you created one via the previous notebook.

In [ ]:
phenotype_filename = f'{bucket}/data/genomics_mt_phenotypes.tsv'
phenotype_filename

- If you did not create your own phenotype file, you can uncomment the following line to read the copy available in the featured workspace bucket.
- This is just an example phenotype file, please create your own phenotype file for a project.

In [ ]:
phenotypes = (hl.import_table(phenotype_filename,
                              types={'person_id':hl.tstr},
                              impute=True,
                              key='person_id')
             )

Before performing a series of variant QC, we need to filter the Hail MatrixTable to only keep samples with phenotype values. We will use the function `semi_join_cols` to keep only samples in the pre-generated phenotype file.

In [ ]:
mt = mt.semi_join_cols(phenotypes)
#mt.count()

## Link phenotypic data with genomic data

Before running a GWAS, we need to annotate the genomic data with the phenotype data. We will use the function `annotate_cols` to perform this step.

In [ ]:
mt = mt.annotate_cols(pheno = phenotypes[mt.s])

# Pre-process the genomic data

Pre-processing includes sample QC and variant QC. We will follow the protocol with subtle modification in this published paper [A tutorial on conducting genome-wide association studies: Quality control and statistical analysis]( https://onlinelibrary.wiley.com/doi/10.1002/mpr.1608): 
- inconsistencies in assigned and genetic sex of subjects (see sex discrepancy), 
- minor allele frequency (MAF), 
- deviations from Hardy–Weinberg equilibrium (HWE), 
- heterozygosity rate, 
- relatedness, 
- ethnic outliers (population stratification).

In [ ]:
mt.count()

Below includes sample code to restrict the genetic variants to a specified range

In [ ]:
# test_intervals = ['chrX:68112092-68112093','chrX:10198158-10198159','chrX:118782557-118782558'
#  ,'chrX:79170974-79170975' ,'chrX:21843316-21843317', 'chrX:40063106-40063107', 
#                  'chrX:53434412-53434413', 'chrX:110688628-110688629', 'chrX:101259997-101259998'] 

# test_intervals = ['chrY:13470103-13470104']

# mt = hl.filter_intervals(
#     mt,
#     [hl.parse_locus_interval(x,)
#      for x in test_intervals])

## Sex discrepancy

Sex concordance is part of the *All of Us* upstream genomic data quality control process, and all samples have passed the sex concordance check, so we do not need to perform this step here. For more details about the sex concordance check, please refer to the [All of Us Genomic Quality Report]( https://aousupporthelp.zendesk.com/hc/en-us/articles/4617899955092-All-of-Us-Beta-Release-Genomic-Quality-Report-).

## Relatedness

The *All of Us* Data and Research Center provides a list of samples to remove related samples from the full cohort. We will use the function `anti_join_cols` to perform this step. For more details about this list of related samples, please refer to the support article [How the All of Us Genomic data are organized](https://aousupporthelp.zendesk.com/hc/en-us/articles/4614687617556-How-the-All-of-Us-Genomic-data-are-organized).

In [ ]:
related_samples_path = "gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/relatedness/relatedness_flagged_samples.tsv"

In [ ]:
related_remove = hl.import_table(related_samples_path,
                                 types={"sample_id":"tstr"},
                                key="sample_id")

#related_remove.count()

In [ ]:
mt = mt.anti_join_cols(related_remove)
#mt.count()

## Population stratification

The *All of Us* Data and Research Center provides genetic predicted ancestry including the principal components (PCs) for the WGS data. We will incorporate thes PCs into the Hail MatrixTable and set them as covariate to take population stratification into account during model building. We will read the ancestry table and annoate the Hail MatrixTable with the ancestry table in this step.

For more information about the ancestry prediction table, please refer to this support article [How the All of Us Genomic data are organized](https://aousupporthelp.zendesk.com/hc/en-us/articles/4614687617556-How-the-All-of-Us-Genomic-data-are-organized).

In [ ]:
ancestry_pred_path = "gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv"

In [ ]:
ancestry_pred = hl.import_table(ancestry_pred_path,
                               key="research_id", 
                               impute=True, 
                               types={"research_id":"tstr","pca_features":hl.tarray(hl.tfloat)})

In [ ]:
mt = mt.annotate_cols(ancestry_pred = ancestry_pred[mt.s])

Below includes code to filter the population by majority predicted ancestry.  When running a multiethnic GWAS, results can be slightly improved by analyzing each ethinity seperately and meta-analyzing the results.

In [ ]:
# mt_eur = mt.filter_cols(mt.ancestry_pred.ancestry_pred == "eur")
# # mt_pheno_eur.col.describe()
# mt_orig = mt
# mt = mt_eur

## Heterozygosity rate

Allele Balance (AB) in the “Hard Threshold Filters” is part of the *All of Us* upstream data quality control processes. It marks all the variants that do not meet the threshold AB>=0.2 for heterozygotes as `NO_HQ_GENOTYPES` in the `filters` field. There are other criteria to flag specific variants from a callset which results in different values in the `filters` field. 

The MatrixTables for small regions have removed variants that don't pass the filters. For more details about the `filters` field, please please refer to this support article [How the All of Us Genomic data are organized](https://aousupporthelp.zendesk.com/hc/en-us/articles/4614687617556-How-the-All-of-Us-Genomic-data-are-organized).

## Minor allele frequency (MAF)

Since we have removed some samples, we need to recompute common variant statistics for the variant quality control metrics.

In [ ]:
mt = hl.variant_qc(mt)

The MAF threshold should depend on your sample size; larger samples can use lower MAF thresholds. For large (N = 100,000) vs. moderate samples (N = 10,000), 0.01 and 0.05 are commonly used as MAF threshold, respectively.

In [ ]:
mt = mt.filter_rows(hl.min(mt.variant_qc.AF) > 0.05, keep = True)
#mt.count()

## Deviations from Hardy–Weinberg equilibrium (HWE)

We will use 1e−10 as HWE p value threshold in the example.

This was removed, as multi-ethnic cohorts will violate HWE

In [ ]:
#mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-20, keep = True)
#mt.count()

# Genome-Wide Association Study (GWAS)

Once the data has been pre-processed, we are ready to move on to modeling. 

We will run a logistic regression with:
- diabetes as the dependent variable; 
- n_alt_alleles from the genomic data as our independent variable X; and
- sex at birth and ethnicity as covariates;
- set the first three PC features as covariates.

We will start by putting all of the covariates in a variable. Note: the demographics data were integrated in the MatrixTable **mt** as a column field named **pheno**, PCA features were in the column field **ancestry_pred**.

Please define your own covariates in real analysis.

In [ ]:
covariates = [1.0, mt.pheno.is_male, mt.pheno.age_yrs,
             mt.ancestry_pred.pca_features[0], 
              mt.ancestry_pred.pca_features[1], 
              mt.ancestry_pred.pca_features[2],
              mt.ancestry_pred.pca_features[3],
              mt.ancestry_pred.pca_features[4],
              mt.ancestry_pred.pca_features[5],
              mt.ancestry_pred.pca_features[6],
              mt.ancestry_pred.pca_features[7],
              mt.ancestry_pred.pca_features[8],
              mt.ancestry_pred.pca_features[9],
              mt.ancestry_pred.pca_features[10], 
              mt.ancestry_pred.pca_features[11], 
              mt.ancestry_pred.pca_features[12], 
              mt.ancestry_pred.pca_features[13],
              mt.ancestry_pred.pca_features[14],
              mt.ancestry_pred.pca_features[15]]

In [ ]:
log_reg = hl.logistic_regression_rows(
    test='wald',
    y=mt.pheno.has_mt,
    x=mt.GT.n_alt_alleles(),
    covariates=covariates
)

<div class="alert alert-block alert-info">
    <b>Tip:</b> If you want to check the contents of <i>log_reg</i>, you can use
<pre>
log_reg.describe()
</pre>
</div>

We can also use the function `export` to save the GWAS results to the bucket for downstream analysis if needed. It is recommended to save large files with a `.bgz` extension to save time and space. Please check [Hail Table.export](https://hail.is/docs/0.2/hail.Table.html#hail.Table.export) for more detailes. 

Note that this method will export results to a text file. All values will be string in the file.

<div class="alert alert-block alert-info">
<b>Tip:</b> We first flatten the nested fields using function flatten, and then use the function `export` to save results to workspace bucket.


<pre>
log_reg = log_reg.flatten()

log_reg_save_path = f'{bucket}/data/log_reg.tsv.bgz'

log_reg.export(log_reg_save_path)
</pre>
To reuse them at a later time:
<pre>
gwas_result = hl.import_table(log_reg_save_path, types={"locus":hl.tlocus(reference_genome='GRCh38'),"alleles": hl.tarray(hl.tstr), "beta": hl.tfloat64, "p_value": hl.tfloat64, "fit.n_iterations": hl.tint32})
</pre>
</div>


In [ ]:
log_reg = log_reg.flatten()

log_reg_save_path = f'{bucket}/data/log_reg.tsv.bgz'

log_reg.export(log_reg_save_path)

# References:

- [Hail Documentation](https://hail.is/docs/0.2/tutorials-landing.html) 
- [Hail tutorials](https://app.terra.bio/#workspaces/help-gatk/Hail-Notebook-Tutorials) 
- [A demo workspace for working with gnomAD data in Terra](https://terra.bio/a-demo-workspace-for-working-with-gnomad-data-in-terra/) 
    - [Hail notebook](https://app.terra.bio/#workspaces/terra-outreach/DEMO-Working-with-gnomAD)
- [Copy files/objects to/from a bucket](https://cloud.google.com/storage/docs/gsutil/commands/cp) 
- [A tutorial on conducting genome-wide association studies: Quality control and statistical analysis]( https://onlinelibrary.wiley.com/doi/10.1002/mpr.1608) 